<a href="https://colab.research.google.com/github/SanjayJanardhan-89/ApacheSparkHandsOn/blob/main/Certification/Chapter_5_Advanced_Operations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup Pyspark


In [1]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

import os
import sys
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"


import findspark
findspark.init()
findspark.find()

import pyspark
from pyspark.sql import SparkSession

spark= SparkSession \
       .builder \
       .appName("OurSparkApp") \
       .getOrCreate()

spark

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:5 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:6 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Get:9 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,848 kB]
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,245 kB]
Hit:12 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Fetched 4,222 kB in 2s (2,796 kB/s)
Reading package lists... Done
Building dependency tree... Done
Rea

In [2]:
salary_data = [("John", "Field-eng", 3500),
    ("Michael", "Field-eng", 4500),
    ("Robert", None, 4000),
    ("Maria", "Finance", 3500),
    ("John", "Sales", 3000),
    ("Kelly", "Finance", 3500),
    ("Kate", "Finance", 3000),
    ("Martin", None, 3500),
    ("Kiran", "Sales", 2200),
    ("Michael", "Field-eng", 4500)
  ]
columns= ["Employee", "Department", "Salary"]
salary_data = spark.createDataFrame(data = salary_data, schema = columns)
salary_data.printSchema()
salary_data.show()

root
 |-- Employee: string (nullable = true)
 |-- Department: string (nullable = true)
 |-- Salary: long (nullable = true)

+--------+----------+------+
|Employee|Department|Salary|
+--------+----------+------+
|    John| Field-eng|  3500|
| Michael| Field-eng|  4500|
|  Robert|      NULL|  4000|
|   Maria|   Finance|  3500|
|    John|     Sales|  3000|
|   Kelly|   Finance|  3500|
|    Kate|   Finance|  3000|
|  Martin|      NULL|  3500|
|   Kiran|     Sales|  2200|
| Michael| Field-eng|  4500|
+--------+----------+------+



In [3]:
salary_data.groupBy('Department')

GroupedData[grouping expressions: [Department], value: [Employee: string, Department: string ... 1 more field], type: GroupBy]

In [4]:
salary_data.groupBy().avg().show()

+-----------+
|avg(Salary)|
+-----------+
|     3520.0|
+-----------+



In [5]:
salary_data.groupBy('Department').avg().show()

+----------+------------------+
|Department|       avg(Salary)|
+----------+------------------+
|     Sales|            2600.0|
| Field-eng| 4166.666666666667|
|      NULL|            3750.0|
|   Finance|3333.3333333333335|
+----------+------------------+



In [6]:
from pyspark.sql.functions import lit
salary_data = salary_data.withColumn('AnotherCol', lit(100))

In [7]:
salary_data.groupBy('Department').avg().show()

+----------+------------------+---------------+
|Department|       avg(Salary)|avg(AnotherCol)|
+----------+------------------+---------------+
|     Sales|            2600.0|          100.0|
| Field-eng| 4166.666666666667|          100.0|
|      NULL|            3750.0|          100.0|
|   Finance|3333.3333333333335|          100.0|
+----------+------------------+---------------+



In [8]:
salary_data = salary_data.drop('AnotherCol')

In [9]:
salary_data.show()

+--------+----------+------+
|Employee|Department|Salary|
+--------+----------+------+
|    John| Field-eng|  3500|
| Michael| Field-eng|  4500|
|  Robert|      NULL|  4000|
|   Maria|   Finance|  3500|
|    John|     Sales|  3000|
|   Kelly|   Finance|  3500|
|    Kate|   Finance|  3000|
|  Martin|      NULL|  3500|
|   Kiran|     Sales|  2200|
| Michael| Field-eng|  4500|
+--------+----------+------+



In [10]:
from pyspark.sql.functions import *

(salary_data
  .groupBy("Department").sum("salary")
  .withColumn("sum(salary)", round(col("sum(salary)"),4))
  .withColumnRenamed("sum(salary)", "sum_salary")
  .orderBy(desc("sum_salary"))
  .show()
)

+----------+----------+
|Department|sum_salary|
+----------+----------+
| Field-eng|     12500|
|   Finance|     10000|
|      NULL|      7500|
|     Sales|      5200|
+----------+----------+



In [11]:
salary_data_with_id = [(1, "John", "Field-eng", 3500), \
    (2, "Robert", "Sales", 4000), \
    (3, "Maria", "Finance", 3500), \
    (4, "Michael", "Sales", 3000), \
    (5, "Kelly", "Finance", 3500), \
    (6, "Kate", "Finance", 3000), \
    (7, "Martin", "Finance", 3500), \
    (8, "Kiran", "Sales", 2200), \
  ]
columns= ["ID", "Employee", "Department", "Salary"]
salary_data_with_id = spark.createDataFrame(data = salary_data_with_id, schema = columns)
salary_data_with_id.show()

employee_data = [(1, "NY", "M"), \
    (2, "NC", "M"), \
    (3, "NY", "F"), \
    (4, "TX", "M"), \
    (5, "NY", "F"), \
    (6, "AZ", "F") \
  ]
columns= ["ID", "State", "Gender"]
employee_data = spark.createDataFrame(data = employee_data, schema = columns)
employee_data.show()

+---+--------+----------+------+
| ID|Employee|Department|Salary|
+---+--------+----------+------+
|  1|    John| Field-eng|  3500|
|  2|  Robert|     Sales|  4000|
|  3|   Maria|   Finance|  3500|
|  4| Michael|     Sales|  3000|
|  5|   Kelly|   Finance|  3500|
|  6|    Kate|   Finance|  3000|
|  7|  Martin|   Finance|  3500|
|  8|   Kiran|     Sales|  2200|
+---+--------+----------+------+

+---+-----+------+
| ID|State|Gender|
+---+-----+------+
|  1|   NY|     M|
|  2|   NC|     M|
|  3|   NY|     F|
|  4|   TX|     M|
|  5|   NY|     F|
|  6|   AZ|     F|
+---+-----+------+



In [12]:
salary_data_with_id.join(employee_data, salary_data_with_id.ID == employee_data.ID).show()

+---+--------+----------+------+---+-----+------+
| ID|Employee|Department|Salary| ID|State|Gender|
+---+--------+----------+------+---+-----+------+
|  1|    John| Field-eng|  3500|  1|   NY|     M|
|  2|  Robert|     Sales|  4000|  2|   NC|     M|
|  3|   Maria|   Finance|  3500|  3|   NY|     F|
|  4| Michael|     Sales|  3000|  4|   TX|     M|
|  5|   Kelly|   Finance|  3500|  5|   NY|     F|
|  6|    Kate|   Finance|  3000|  6|   AZ|     F|
+---+--------+----------+------+---+-----+------+



In [13]:
salary_data_with_id.join(employee_data, salary_data_with_id.ID == employee_data.ID, "full").show()

+---+--------+----------+------+----+-----+------+
| ID|Employee|Department|Salary|  ID|State|Gender|
+---+--------+----------+------+----+-----+------+
|  1|    John| Field-eng|  3500|   1|   NY|     M|
|  2|  Robert|     Sales|  4000|   2|   NC|     M|
|  3|   Maria|   Finance|  3500|   3|   NY|     F|
|  4| Michael|     Sales|  3000|   4|   TX|     M|
|  5|   Kelly|   Finance|  3500|   5|   NY|     F|
|  6|    Kate|   Finance|  3000|   6|   AZ|     F|
|  7|  Martin|   Finance|  3500|NULL| NULL|  NULL|
|  8|   Kiran|     Sales|  2200|NULL| NULL|  NULL|
+---+--------+----------+------+----+-----+------+



In [14]:
salary_data_with_id.crossJoin(employee_data).show()

+---+--------+----------+------+---+-----+------+
| ID|Employee|Department|Salary| ID|State|Gender|
+---+--------+----------+------+---+-----+------+
|  1|    John| Field-eng|  3500|  1|   NY|     M|
|  1|    John| Field-eng|  3500|  2|   NC|     M|
|  1|    John| Field-eng|  3500|  3|   NY|     F|
|  2|  Robert|     Sales|  4000|  1|   NY|     M|
|  2|  Robert|     Sales|  4000|  2|   NC|     M|
|  2|  Robert|     Sales|  4000|  3|   NY|     F|
|  3|   Maria|   Finance|  3500|  1|   NY|     M|
|  3|   Maria|   Finance|  3500|  2|   NC|     M|
|  3|   Maria|   Finance|  3500|  3|   NY|     F|
|  4| Michael|     Sales|  3000|  1|   NY|     M|
|  4| Michael|     Sales|  3000|  2|   NC|     M|
|  4| Michael|     Sales|  3000|  3|   NY|     F|
|  1|    John| Field-eng|  3500|  4|   TX|     M|
|  1|    John| Field-eng|  3500|  5|   NY|     F|
|  1|    John| Field-eng|  3500|  6|   AZ|     F|
|  2|  Robert|     Sales|  4000|  4|   TX|     M|
|  2|  Robert|     Sales|  4000|  5|   NY|     F|


In [15]:
salary_data_with_id_2 = \
 [
              (1, "John", "Field-eng", 3500), \
              (2, "Robert", "Sales", 4000), \
              (3, "Aliya", "Finance", 3500), \
              (4, "Nate", "Sales", 3000), \
 ]
columns2= ["ID", "Employee", "Department", "Salary"]
salary_data_with_id_2 = spark.createDataFrame(data = salary_data_with_id_2, schema = columns2)
salary_data_with_id_2.printSchema()
salary_data_with_id_2.show(truncate=False)

root
 |-- ID: long (nullable = true)
 |-- Employee: string (nullable = true)
 |-- Department: string (nullable = true)
 |-- Salary: long (nullable = true)

+---+--------+----------+------+
|ID |Employee|Department|Salary|
+---+--------+----------+------+
|1  |John    |Field-eng |3500  |
|2  |Robert  |Sales     |4000  |
|3  |Aliya   |Finance   |3500  |
|4  |Nate    |Sales     |3000  |
+---+--------+----------+------+



In [16]:
salary_data_with_id.union(salary_data_with_id_2).show()

+---+--------+----------+------+
| ID|Employee|Department|Salary|
+---+--------+----------+------+
|  1|    John| Field-eng|  3500|
|  2|  Robert|     Sales|  4000|
|  3|   Maria|   Finance|  3500|
|  4| Michael|     Sales|  3000|
|  5|   Kelly|   Finance|  3500|
|  6|    Kate|   Finance|  3000|
|  7|  Martin|   Finance|  3500|
|  8|   Kiran|     Sales|  2200|
|  1|    John| Field-eng|  3500|
|  2|  Robert|     Sales|  4000|
|  3|   Aliya|   Finance|  3500|
|  4|    Nate|     Sales|  3000|
+---+--------+----------+------+



In [17]:
salary_data_with_id.intersect(salary_data_with_id_2).show()

+---+--------+----------+------+
| ID|Employee|Department|Salary|
+---+--------+----------+------+
|  2|  Robert|     Sales|  4000|
|  1|    John| Field-eng|  3500|
+---+--------+----------+------+



In [18]:
salary_data_with_id.union(salary_data_with_id_2).distinct().show()

+---+--------+----------+------+
| ID|Employee|Department|Salary|
+---+--------+----------+------+
|  4| Michael|     Sales|  3000|
|  3|   Maria|   Finance|  3500|
|  2|  Robert|     Sales|  4000|
|  1|    John| Field-eng|  3500|
|  8|   Kiran|     Sales|  2200|
|  6|    Kate|   Finance|  3000|
|  7|  Martin|   Finance|  3500|
|  5|   Kelly|   Finance|  3500|
|  3|   Aliya|   Finance|  3500|
|  4|    Nate|     Sales|  3000|
+---+--------+----------+------+



In [19]:
(employee_data
 .write
 .format('csv')
 .mode('overwrite')
 .option("header", True)
 .save('salary_data.csv')
 )


In [20]:
spark.read.format('csv').load('salary_data.csv').show()

+---+-----+------+
|_c0|  _c1|   _c2|
+---+-----+------+
| ID|State|Gender|
|  1|   NY|     M|
|  2|   NC|     M|
|  3|   NY|     F|
| ID|State|Gender|
|  4|   TX|     M|
|  5|   NY|     F|
|  6|   AZ|     F|
+---+-----+------+



In [21]:
(spark.read
 .format('csv')
 .option("header", True)
 .load('salary_data.csv').show()
)

+---+-----+------+
| ID|State|Gender|
+---+-----+------+
|  1|   NY|     M|
|  2|   NC|     M|
|  3|   NY|     F|
|  4|   TX|     M|
|  5|   NY|     F|
|  6|   AZ|     F|
+---+-----+------+



In [22]:
from pyspark.sql.types import *
filePath = '\salary_data.csv'
columns= ["ID", "State", "Gender"]
schema = StructType([
      StructField("ID", IntegerType(),True),
  StructField("State",  StringType(),True),
  StructField("Gender",  StringType(),True)
])


read_data = spark.read.format("csv").option("header","true").schema(schema).load(filePath)
read_data.show()


+---+-----+------+
| ID|State|Gender|
+---+-----+------+
|  1|   NY|     M|
|  2|   NC|     M|
|  3|   NY|     F|
|  4|   TX|     M|
|  5|   NY|     F|
|  6|   AZ|     F|
+---+-----+------+



In [23]:
salary_data_with_id.write.mode("append").format("parquet").save("salary_data.parquet")

In [24]:
spark.read.format("parquet").load("salary_data.parquet").show()

+---+--------+----------+------+
| ID|Employee|Department|Salary|
+---+--------+----------+------+
|  1|    John| Field-eng|  3500|
|  2|  Robert|     Sales|  4000|
|  3|   Maria|   Finance|  3500|
|  4| Michael|     Sales|  3000|
|  1|    John| Field-eng|  3500|
|  2|  Robert|     Sales|  4000|
|  3|   Maria|   Finance|  3500|
|  4| Michael|     Sales|  3000|
|  1|    John| Field-eng|  3500|
|  2|  Robert|     Sales|  4000|
|  3|   Maria|   Finance|  3500|
|  4| Michael|     Sales|  3000|
|  5|   Kelly|   Finance|  3500|
|  6|    Kate|   Finance|  3000|
|  7|  Martin|   Finance|  3500|
|  8|   Kiran|     Sales|  2200|
|  5|   Kelly|   Finance|  3500|
|  6|    Kate|   Finance|  3000|
|  7|  Martin|   Finance|  3500|
|  8|   Kiran|     Sales|  2200|
+---+--------+----------+------+
only showing top 20 rows



In [25]:
salary_data_with_id.write.mode("append").format("orc").save("salary_data.orc")
spark.read.format("orc").load("salary_data.orc").show()

+---+--------+----------+------+
| ID|Employee|Department|Salary|
+---+--------+----------+------+
|  1|    John| Field-eng|  3500|
|  2|  Robert|     Sales|  4000|
|  3|   Maria|   Finance|  3500|
|  4| Michael|     Sales|  3000|
|  1|    John| Field-eng|  3500|
|  2|  Robert|     Sales|  4000|
|  3|   Maria|   Finance|  3500|
|  4| Michael|     Sales|  3000|
|  5|   Kelly|   Finance|  3500|
|  6|    Kate|   Finance|  3000|
|  7|  Martin|   Finance|  3500|
|  8|   Kiran|     Sales|  2200|
|  5|   Kelly|   Finance|  3500|
|  6|    Kate|   Finance|  3000|
|  7|  Martin|   Finance|  3500|
|  8|   Kiran|     Sales|  2200|
+---+--------+----------+------+



# Delta File

In [32]:
salary_data_with_id.write.mode("append").format("delta").save("salary_data.delta")


Py4JJavaError: An error occurred while calling o220.save.
: org.apache.spark.SparkClassNotFoundException: [DATA_SOURCE_NOT_FOUND] Failed to find the data source: delta. Please find packages at `https://spark.apache.org/third-party-projects.html`.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.dataSourceNotFoundError(QueryExecutionErrors.scala:725)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSource(DataSource.scala:647)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSourceV2(DataSource.scala:697)
	at org.apache.spark.sql.DataFrameWriter.lookupV2Provider(DataFrameWriter.scala:873)
	at org.apache.spark.sql.DataFrameWriter.saveInternal(DataFrameWriter.scala:260)
	at org.apache.spark.sql.DataFrameWriter.save(DataFrameWriter.scala:243)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at java.base/jdk.inte